# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

*By*: **Fernando Emmanuel Quintana**

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [3]:
from sklearn.externals import joblib
eclf = joblib.load('./sentiment.pkl') 

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [16]:
!pip uninstall watson-machine-learning-client -y

Skipping watson-machine-learning-client as it is not installed.


In [17]:
# Install the WML client API
!pip install watson-machine-learning-client-V4

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [5]:
wml_credentials = {
  "apikey": "JXJ1YkxyxPWxNIi4zI6u46TqRNps3HKLh1-Y9fSUFlDG",
  "iam_apikey_description": "Auto-generated for key 5a9aab5a-1a17-4d3b-8a73-4a65157f3b89",
  "iam_apikey_name": "credentials-ml-service-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/9b5afd1de1df4dcba0e7c996eacf03b4::serviceid:ServiceId-690d918d-471d-4d53-98ec-dd9ec849979d",
  "instance_id": "9d469252-fde5-4981-9071-5f069773f8fb",
  "url": "https://us-south.ml.cloud.ibm.com"
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [6]:
client = WatsonMachineLearningAPIClient(wml_credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [7]:
model_props = {
    client.repository.ModelMetaNames.NAME: "MovieReviewsClassification",
    client.repository.ModelMetaNames.DESCRIPTION: "Movies review NLP Classifier",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_0.20",
    client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.20-py3" 
}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
pipeline = make_pipeline(vectorizer, eclf)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth..., random_state=None))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [14]:
published_model = client.repository.store_model(
    model=pipeline,
    meta_props=model_props,
    training_data=X_train,
    training_target=y_train
)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [15]:
published_model_uid = client.repository.get_model_uid(published_model)
print('Published Model UID: ', published_model_uid)

Published Model UID:  99b8812a-933f-40c2-9b3b-df891cf4103a


In [16]:
models_details = client.repository.list_models()

------------------------------------  --------------------------  ------------------------  -----------------
GUID                                  NAME                        CREATED                   TYPE
99b8812a-933f-40c2-9b3b-df891cf4103a  MovieReviewsClassification  2019-09-03T16:39:08.541Z  scikit-learn_0.20
------------------------------------  --------------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [17]:
stored_model = client.repository.load(published_model_uid)
test_predictions = stored_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [18]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print('Roc_auc score: ', roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

Roc_auc score:  0.8574999999999999
              precision    recall  f1-score   support

           0       0.83      0.90      0.86       200
           1       0.89      0.81      0.85       200

   micro avg       0.86      0.86      0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400



*By*: **Fernando Emmanuel Quintana**